# TODO

* Add actions:
  - Delete/edit feedback option
  - Switch students
* Hook up database
* Make a dashboard layout
* Deploy

In [1]:
from itertools import starmap
from collections import namedtuple
import random
import time

from IPython import display
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox, Textarea, Image
import pandas
import jinja2
from pony import orm

import db

In [8]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 3,
      source: availableTags
    });
  } );
""")

prevent_unselect = """
window.onmousedown = function (e) {
    var el = e.target;
    if (el.tagName.toLowerCase() == 'option' && el.parentNode.hasAttribute('multiple')) {
        e.preventDefault();

        // toggle selection
        if (el.hasAttribute('selected')) el.removeAttribute('selected');
        else el.setAttribute('selected', '');

        // hack to correct buggy behavior
        var select = el.parentNode.cloneNode(true);
        el.parentNode.parentNode.replaceChild(select, el.parentNode);
    }
}
"""

def add_option(target):
    def wrapper(sender):
        # add to database
        with orm.db_session:
            if not db.FeedbackOption.exists(text=sender.value):
                db.FeedbackOption(text=sender.value, problems=db.Problem[problem_id_global])
            
        with target.hold_trait_notifications():
            target.options = target.options + [sender.value]
        sender.value = ''
    return wrapper

In [9]:
### Model -- grab data from database

# grab data from database
students = list(starmap('{} ({} {})'.format, orm.select((s.id, s.first_name, s.last_name) for s in db.Student)))
feedback_options = [f.text for f in db.FeedbackOption.select()]

problem_id_global = 1
submission_id_global = 1

In [10]:
### All the individual widgets

prev_stud = Button(icon='fa-fast-backward', description=' previous')
next_stud = Button(icon='fa-fast-forward', description=' submit & next', button_style='warning')
next_stud_in_list = Button(icon='fa-forward', description=' next')
jump_to_stud = Text(description='Jump to', placeholder='# or name')

previous_problem = Button(icon='fa-arrow-circle-up', description=' previous problem')
next_problem = Button(icon='fa-arrow-circle-down', description=' next problem')

options_feedback = SelectMultiple(name='feedback', options=['Everything correct', 'No solution provided'])
new_option = Text(placeholder='New feedback option')
new_option.on_submit(add_option(options_feedback))
student_specific = Textarea(placeholder='Student-specific feedback')
#with open('test_exam/sample_data.jpg', 'rb') as f:
#    data = f.read()
#answer = Image(value=data, format='png', width='100%')


In [11]:
### Widget layout

widgets = VBox([
        HBox([prev_stud, next_stud, next_stud_in_list, jump_to_stud]),
        HBox([next_problem, previous_problem]),
        HBox([VBox([options_feedback, new_option]), student_specific]),
        #answer,
])

In [12]:
### Widget rendering

# display widgets
display.display(widgets)

# apply javascript kludges
selector = "placeholder='{}'".format(jump_to_stud.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=students)))
display.display_javascript(prevent_unselect)